In [1]:
import pandas as pd
import numpy as np
!conda install -c anaconda certifi

Solving environment: ...working... done

# All requested packages already installed.



In [2]:
!pip install --upgrade certifi

In [3]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
#PRIMERA PARTE

In [4]:
df_toronto = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1008658788')
df_toronto

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 ..          ...               ...   
 175         M5Z      Not assigned   
 176         M6Z      Not assigned   
 177         M7Z      Not assigned   
 178         M8Z         Etobicoke   
 179         M9Z      Not assigned   
 
                                          Neighbourhood  
 0                                         Not assigned  
 1                                         Not assigned  
 2                                            Parkwoods  
 3                                     Victoria Village  
 4                            Regent Park, Harbourfront  
 ..                                                 ...  
 175                                       Not assigned  
 176                                       Not assigned  
 177                

In [5]:
print(f'Número total de tablas: {len(df_toronto)}')

Número total de tablas: 3


In [6]:
df_code = df_toronto[0]
df_code.head(12)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [29]:
df_code2 = df_code[df_code['Borough']!= 'Not assigned']
df_code2 = df_code2.reset_index(drop=True)
df_code2.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [36]:
df_code2[df_code2['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [31]:
print('El dataframe tiene {} filas'.format(df_code2.shape[0]), 'y {} columnas'.format(df_code2.shape[1]))
df_code2.to_csv('df_toronto_v1.csv', index=True)

El dataframe tiene 103 filas y 3 columnas


In [ ]:
##SEGUNDA PARTE

In [32]:
coordenadas = pd.read_csv('Geospatial_Coordinates.csv')
coordenadas.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
coord_list=pd.merge(df_code,coordenadas,on="Postal Code")
coord_list.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [ ]:
##TERCERA PARTE

In [34]:
#Filtrar solo los datos que contentan a Toronto
Toronto_df = coord_list[coord_list['Borough'].str.contains('Toronto')]
Toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [35]:
#Barrios por municipio:
Toronto_df.groupby("Borough").size().sort_values(ascending=False)

Borough
Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
dtype: int64

In [23]:
import folium
!pip install geopy
from geopy.geocoders import Nominatim

In [24]:
#Coordenadas necesarias para realizar el mapa
address = 'TORONTO,CA'
geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [37]:
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generar el mapa alrededor de TORONTO

# Añadir TORONTO como una marca circular roja
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(Toronto_map)

# Los barrios figuran con marcas azules
for lat, lng, label in zip(Toronto_df.Latitude, Toronto_df.Longitude, Toronto_df.Neighbourhood):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(Toronto_map)

# mostrar mapa
Toronto_map